In [2]:
import org.apache.spark.sql.SparkSession
val sc = SparkSession.builder().getOrCreate()
sc.version

sc = org.apache.spark.sql.SparkSession@44c82c5c


2.4.3

### 데이터 불러오기

In [3]:
val df = sc.read.format("csv").option("header", "true").option("inferSchema", "true")
            .load("./by-day/2010-12-10.csv")
df.printSchema()
df.createOrReplaceTempView("dfTable")

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



df = [InvoiceNo: string, StockCode: string ... 6 more fields]


[InvoiceNo: string, StockCode: string ... 6 more fields]

In [82]:
df.show(10)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   538172|    21562|HAWAIIAN GRASS SK...|      12|2010-12-10 09:33:00|     1.25|   15805.0|United Kingdom|
|   538172|    79321|       CHILLI LIGHTS|       8|2010-12-10 09:33:00|     4.95|   15805.0|United Kingdom|
|   538172|    22041|"RECORD FRAME 7""...|      12|2010-12-10 09:33:00|     2.55|   15805.0|United Kingdom|
|   538172|   84558A|3D DOG PICTURE PL...|      12|2010-12-10 09:33:00|     2.95|   15805.0|United Kingdom|
|   538172|    22952|60 CAKE CASES VIN...|      24|2010-12-10 09:33:00|     0.55|   15805.0|United Kingdom|
|   538172|    22910|PAPER CHAIN KIT V...|      24|2010-12-10 09:33:00|     2.95|   15805.0|United Kingdom|
|   538172|    21098|CHRISTM

### 2. 스파크 데이터 타입으로 변환하기

프로그래밍 언어의 고유 데이터 타입을 lit 함수를 사용해 스파크 데이터 타입으로 변환함

In [84]:
import org.apache.spark.sql.functions.lit

df.select(lit(5), lit("five"), lit(5.0)).show(10)

+---+----+---+
|  5|five|5.0|
+---+----+---+
|  5|five|5.0|
|  5|five|5.0|
|  5|five|5.0|
|  5|five|5.0|
|  5|five|5.0|
|  5|five|5.0|
|  5|five|5.0|
|  5|five|5.0|
|  5|five|5.0|
|  5|five|5.0|
+---+----+---+
only showing top 10 rows



### 3. 불리언 데이터 타입 다루기

불리언은 모든 필터링 작업의 기반이므로 필수적으로 알아야 함. <br>
and, or, true, false로 구성됨. <br>
불리언 구문을 사용해 true 또는 false로 평가되는 논리 문법을 만들고, 논리 문법은 데이터 로우를 필터링할 때 필요조건의 일치(true)와 불일치(false)를 판별하는데 사용됨. <br>

In [5]:
import org.apache.spark.sql.functions.col

df.where(col("InvoiceNo").equalTo(538174))
    .select("InvoiceNo", "Description")
    .show(5, false)

// equalTo는 스칼라에서 === 와 동일하다.
// ==는 reference 비교, ===은 value 비교
df.where(col("InvoiceNo") === (538174))
    .select("InvoiceNo", "Description")
    .show(5, false)

+---------+----------------------------------+
|InvoiceNo|Description                       |
+---------+----------------------------------+
|538174   |SET OF 72 RETROSPOT PAPER  DOILIES|
|538174   |PACK OF 72 RETROSPOT CAKE CASES   |
|538174   |WOODLAND DESIGN  COTTON TOTE BAG  |
|538174   |BIG DOUGHNUT FRIDGE MAGNETS       |
|538174   |RECYCLED PENCIL WITH RABBIT ERASER|
+---------+----------------------------------+
only showing top 5 rows

+---------+----------------------------------+
|InvoiceNo|Description                       |
+---------+----------------------------------+
|538174   |SET OF 72 RETROSPOT PAPER  DOILIES|
|538174   |PACK OF 72 RETROSPOT CAKE CASES   |
|538174   |WOODLAND DESIGN  COTTON TOTE BAG  |
|538174   |BIG DOUGHNUT FRIDGE MAGNETS       |
|538174   |RECYCLED PENCIL WITH RABBIT ERASER|
+---------+----------------------------------+
only showing top 5 rows



In [12]:
// 가장 명확한 방법은 문자열 표현식에 조건절을 명시하는 것.
df.where("InvoiceNo = 538174").show(5, false)
df.where("InvoiceNo <> 538174").show(5, false)

+---------+---------+----------------------------------+--------+-------------------+---------+----------+-------+
|InvoiceNo|StockCode|Description                       |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country|
+---------+---------+----------------------------------+--------+-------------------+---------+----------+-------+
|538174   |21210    |SET OF 72 RETROSPOT PAPER  DOILIES|12      |2010-12-10 09:35:00|1.45     |12471.0   |Germany|
|538174   |21212    |PACK OF 72 RETROSPOT CAKE CASES   |24      |2010-12-10 09:35:00|0.55     |12471.0   |Germany|
|538174   |21578    |WOODLAND DESIGN  COTTON TOTE BAG  |24      |2010-12-10 09:35:00|2.25     |12471.0   |Germany|
|538174   |21700    |BIG DOUGHNUT FRIDGE MAGNETS       |72      |2010-12-10 09:35:00|0.85     |12471.0   |Germany|
|538174   |16235    |RECYCLED PENCIL WITH RABBIT ERASER|60      |2010-12-10 09:35:00|0.21     |12471.0   |Germany|
+---------+---------+----------------------------------+--------+---------------

and, or 메서드는 사용해서 불리언 표현식을 여러 부분으로 지정할 수 있다. <br>
불리언 표현식을 사용하는 경우 항상 모든 표현식을 and 메서드로 묶어 차례대로 필터를 적용함. <br>
<br>
스파크는 내부적으로 and 구문을 필터 사이에 추가해 모든 필터를 하나의 문장으로 변환함. functional lan 이기 떄문 <br>
 - 그후 동시에 모든 필터를 처리한다.
차례로 조건을 나열하면 이해하기 쉽고 읽기도 편해짐.
 - 반면 or 구문을 사용할 때는 반드시 동일한 구문에 조건을 정의해야 함.

In [16]:
// or 구문을 적용하기 위해 priceFilter, descripFilter를 만들어 or 메서드를 사용함.
val priceFilter = col("UnitPrice") > 600
val descripFilter = col("Description").contains("POSTAGE")
df.where(col("StockCode").isin("DOT")).where(priceFilter.or(descripFilter)).show()

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|   Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|   538177|      DOT|DOTCOM POSTAGE|       1|2010-12-10 09:51:00|   847.42|      null|United Kingdom|
|   538349|      DOT|DOTCOM POSTAGE|       1|2010-12-10 14:59:00|   907.47|      null|United Kingdom|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+



priceFilter = (UnitPrice > 600)
descripFilter = contains(Description, POSTAGE)


contains(Description, POSTAGE)

In [19]:
// 불리언 표현식을 필터링 조건에만 사용하는 것이 아니며, 아래와 같이 사용할수 있다.
val DOTCodeFilter = col("StockCode") === "DOT"
val priceFilter = col("UnitPrice") > 600
val descripFilter = col("Description").contains("POSTAGE")
df.withColumn("isExpensive", DOTCodeFilter.and(priceFilter.or(descripFilter)))
    .where("isExpensive").select("unitPrice", "isExpensive").show(5)

+---------+-----------+
|unitPrice|isExpensive|
+---------+-----------+
|   847.42|       true|
|   907.47|       true|
+---------+-----------+



DOTCodeFilter = (StockCode = DOT)
priceFilter = (UnitPrice > 600)
descripFilter = contains(Description, POSTAGE)


contains(Description, POSTAGE)

In [21]:
// 위와 같이 필터를 반드시 표현식으로 정의할 필요는 없다.
import org.apache.spark.sql.functions.{expr, not, col}

df.withColumn("isExpensive", not(col("UnitPrice").leq(250)))
    .filter("isExpensive")
    .select("Description", "UnitPrice").show(5)

// 아래 코드는 SQL에 익숙한 사람들이 사용하기 좋음.
// SQL을 사용한다고 해서 성능 저하가 발생하지 않는다.
df.withColumn("isExpensive", expr("NOT UnitPrice <= 250"))
    .filter("isExpensive")
    .select("Description", "UnitPrice").show(5)

+--------------+---------+
|   Description|UnitPrice|
+--------------+---------+
|DOTCOM POSTAGE|   847.42|
|DOTCOM POSTAGE|   907.47|
+--------------+---------+



In [12]:
df.where(col("Description").eqNullSafe("Country")).show()

+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
+---------+---------+-----------+--------+-----------+---------+----------+-------+



### 4. 수치형 데이터 타입 다루기

In [23]:
// 간단한 수량을 더하는 작업
import org.apache.spark.sql.functions.{expr, pow}

val fabricatedQuantity = pow(col("Quantity") * col("UnitPrice"), 2) + 5
df.select(expr("CustomerId"), fabricatedQuantity.alias("realQuantity")).show(2)

// SQL 표현
df.selectExpr("CustomerId", "POWER((Quantity * UnitPrice), 2.0) + 5 as realQuantity").show(2)

+----------+------------+
|CustomerId|realQuantity|
+----------+------------+
|   15805.0|       230.0|
|   15805.0|     1573.16|
+----------+------------+
only showing top 2 rows

+----------+------------+
|CustomerId|realQuantity|
+----------+------------+
|   15805.0|       230.0|
|   15805.0|     1573.16|
+----------+------------+
only showing top 2 rows



fabricatedQuantity = (POWER((Quantity * UnitPrice), 2.0) + 5)


(POWER((Quantity * UnitPrice), 2.0) + 5)

In [24]:
// 반올림. 때로는 소수점 자리를 없애기 위해 Integer 데이터 타입으로 형변환하기도 함.
// 내림은 bround 함수를 사용함.
import org.apache.spark.sql.functions.{round, bround}
df.select(round(col("UnitPrice"), 1).alias("rounded"), col("UnitPrice")).show(5)

+-------+---------+
|rounded|UnitPrice|
+-------+---------+
|    1.3|     1.25|
|    5.0|     4.95|
|    2.6|     2.55|
|    3.0|     2.95|
|    0.6|     0.55|
+-------+---------+
only showing top 5 rows



In [25]:
// 반올림, 버림의 예시
import org.apache.spark.sql.functions.lit

df.select(round(lit("2.5")), bround(lit("2.5"))).show(2)

+-------------+--------------+
|round(2.5, 0)|bround(2.5, 0)|
+-------------+--------------+
|          3.0|           2.0|
|          3.0|           2.0|
+-------------+--------------+
only showing top 2 rows



In [30]:
// 두 컬럼 사이의 상관관계 계산
import org.apache.spark.sql.functions.{corr}

df.stat.corr("Quantity", "UnitPrice")
df.select(corr("Quantity", "UnitPrice")).show()

+-------------------------+
|corr(Quantity, UnitPrice)|
+-------------------------+
|     -0.02802551979854...|
+-------------------------+



In [32]:
// 하나 이상의 컬럼에 대한 요약 통계를 계산하는 작업
// 관련 컬럼에 대해 집계, 평균, 표준편차, 최솟값, 최대값 제공
df.select("InvoiceNo", "StockCode", "Quantity", "UnitPrice").describe().show()

+-------+-----------------+------------------+------------------+------------------+
|summary|        InvoiceNo|         StockCode|          Quantity|         UnitPrice|
+-------+-----------------+------------------+------------------+------------------+
|  count|             2758|              2758|              2758|              2758|
|   mean|538266.0758928572|27964.272652726526| 7.359318346627991| 4.650130529369104|
| stddev|76.38281478503671| 17614.08058257242|19.786335248719404|24.187828632489182|
|    min|           538172|             10002|              -145|               0.0|
|    max|          C538362|              POST|               288|            907.47|
+-------+-----------------+------------------+------------------+------------------+



In [36]:
// statFunctions 패키지는 다양한 통계 함수를 제공한다.
val colName = "UnitPrice"
val quantileProbs = Array(0.5)
val relError = 0.05

// approxQuantile을 통해 데이터의 백분위수를 정확하게 계산하거나 근사치를 계산할 수 있음.
df.stat.approxQuantile(colName, quantileProbs, relError)

colName = UnitPrice
quantileProbs = Array(0.5)
relError = 0.05


Array(2.51)

In [37]:
// crosstab을 통해 교차로나 자주 사용하는 항목 쌍을 확인하는 용도의 메서드
// pivot?
df.stat.crosstab("StockCode", "Quantity").show()

+------------------+---+---+---+----+----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|StockCode_Quantity| -1|-11|-12|-144|-145|-15|-18| -2|-20|-21|-24|-28| -3|-30|-36| -4|-47|-48| -6|-96|  1| 10|100|104|106|108| 11|113|115| 12|120|128| 13| 14|144| 15| 16| 17| 18| 19|192|  2| 20| 21|216| 22| 23| 24|240| 25| 26|267| 27|272|288| 29|  3| 30| 31| 32| 33| 35| 36| 38|  4| 40| 45| 47| 48|  5| 50|  6| 60| 64| 65| 68|  7| 70| 72| 75|  8| 80| 82| 84|  9| 96|
+------------------+---+---+---+----+----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+

In [38]:
// monotonically_increasing_id를 통해 모든 로우에 고유 ID 값을 추가할 수 있음.
import org.apache.spark.sql.functions.monotonically_increasing_id
df.select(monotonically_increasing_id()).show(5)

+-----------------------------+
|monotonically_increasing_id()|
+-----------------------------+
|                            0|
|                            1|
|                            2|
|                            3|
|                            4|
+-----------------------------+
only showing top 5 rows



### 5. 문자열 데이터 타입 다루기

In [40]:
// initcap은 주어진 문자열에서 공백으로 나누는 모든 단어의 첫 글자를 대문자로 변경함.
import org.apache.spark.sql.functions.{initcap}

df.select(initcap(col("Description"))).show(5, false)

+-------------------------------+
|initcap(Description)           |
+-------------------------------+
|Hawaiian Grass Skirt           |
|Chilli Lights                  |
|"record Frame 7"" Single Size "|
|3d Dog Picture Playing Cards   |
|60 Cake Cases Vintage Christmas|
+-------------------------------+
only showing top 5 rows



In [41]:
// 소문자로 변경 lower, 대문자로 변경 upper
import org.apache.spark.sql.functions.{lower, upper}

df.select(col("Description"), 
         lower(col("Description")),
         upper(lower(col("Description")))).show(5)

+--------------------+--------------------+-------------------------+
|         Description|  lower(Description)|upper(lower(Description))|
+--------------------+--------------------+-------------------------+
|HAWAIIAN GRASS SK...|hawaiian grass sk...|     HAWAIIAN GRASS SK...|
|       CHILLI LIGHTS|       chilli lights|            CHILLI LIGHTS|
|"RECORD FRAME 7""...|"record frame 7""...|     "RECORD FRAME 7""...|
|3D DOG PICTURE PL...|3d dog picture pl...|     3D DOG PICTURE PL...|
|60 CAKE CASES VIN...|60 cake cases vin...|     60 CAKE CASES VIN...|
+--------------------+--------------------+-------------------------+
only showing top 5 rows



In [21]:
// 공백 제거 함수들
// lpad, rpad는 문자열의 길이보다 작은 숫자를 넘기면 문자열의 오른쪽부터 제거됨.
import org.apache.spark.sql.functions.{lit, ltrim, rtrim, rpad, lpad, trim}

df.select(
    ltrim(lit("   HELLO   ")).as("ltrim"),
    rtrim(lit("   HELLO   ")).as("rtrim"),
    trim(lit("   HELLO   ")).as("trim"),
    lpad(lit("HELLO"), 3, " ").as("lp"),
    rpad(lit("HELLO"), 10, " ").as("rp")
).show(5)

+--------+--------+-----+---+----------+
|   ltrim|   rtrim| trim| lp|        rp|
+--------+--------+-----+---+----------+
|HELLO   |   HELLO|HELLO|HEL|HELLO     |
|HELLO   |   HELLO|HELLO|HEL|HELLO     |
|HELLO   |   HELLO|HELLO|HEL|HELLO     |
|HELLO   |   HELLO|HELLO|HEL|HELLO     |
|HELLO   |   HELLO|HELLO|HEL|HELLO     |
+--------+--------+-----+---+----------+
only showing top 5 rows



In [7]:
// 정규 표현식
// 문자열의 존재 여부를 확인하거나 일치하는 모든 문자열을 치환할 때 사용함.
// regexp_replace

import org.apache.spark.sql.functions.regexp_replace
val simpleColors = Seq("black", "white", "red", "green", "blud")
val regexString = simpleColors.map(_.toUpperCase).mkString("|") // 파이프문자 (|)는 정규표현식에서 or를 뜻함.

df.select(
    regexp_replace(col("Description"), regexString, "COLOR").alias("color_clean"), col("Description")
).show(50, false)

+-----------------------------------+-----------------------------------+
|color_clean                        |Description                        |
+-----------------------------------+-----------------------------------+
|HAWAIIAN GRASS SKIRT               |HAWAIIAN GRASS SKIRT               |
|CHILLI LIGHTS                      |CHILLI LIGHTS                      |
|"RECORD FRAME 7"" SINGLE SIZE "    |"RECORD FRAME 7"" SINGLE SIZE "    |
|3D DOG PICTURE PLAYING CARDS       |3D DOG PICTURE PLAYING CARDS       |
|60 CAKE CASES VINTAGE CHRISTMAS    |60 CAKE CASES VINTAGE CHRISTMAS    |
|PAPER CHAIN KIT VINTAGE CHRISTMAS  |PAPER CHAIN KIT VINTAGE CHRISTMAS  |
|CHRISTMAS TOILET ROLL              |CHRISTMAS TOILET ROLL              |
|"ASSORTED FLOWER COLOUR ""LEIS"""  |"ASSORTED FLOWER COLOUR ""LEIS"""  |
|I CAN ONLY PLEASE ONE PERSON MUG   |I CAN ONLY PLEASE ONE PERSON MUG   |
|HAND WARMER BABUSHKA DESIGN        |HAND WARMER BABUSHKA DESIGN        |
|BLUE HARMONICA IN BOX              |B

simpleColors = List(black, white, red, green, blud)
regexString = BLACK|WHITE|RED|GREEN|BLUD


BLACK|WHITE|RED|GREEN|BLUD

In [14]:
// translate 함수를 사용해 문자 치환
import org.apache.spark.sql.functions.translate

// 연산은 문자 단위로 이루어짐.
df.select(translate(col("Description"), "LEET", "1337"), col("Description")).show(50, false)

+-----------------------------------+-----------------------------------+
|translate(Description, LEET, 1357) |Description                        |
+-----------------------------------+-----------------------------------+
|HAWAIIAN GRASS SKIR7               |HAWAIIAN GRASS SKIRT               |
|CHI11I 1IGH7S                      |CHILLI LIGHTS                      |
|"R3CORD FRAM3 7"" SING13 SIZ3 "    |"RECORD FRAME 7"" SINGLE SIZE "    |
|3D DOG PIC7UR3 P1AYING CARDS       |3D DOG PICTURE PLAYING CARDS       |
|60 CAK3 CAS3S VIN7AG3 CHRIS7MAS    |60 CAKE CASES VINTAGE CHRISTMAS    |
|PAP3R CHAIN KI7 VIN7AG3 CHRIS7MAS  |PAPER CHAIN KIT VINTAGE CHRISTMAS  |
|CHRIS7MAS 7OI137 RO11              |CHRISTMAS TOILET ROLL              |
|"ASSOR73D F1OW3R CO1OUR ""13IS"""  |"ASSORTED FLOWER COLOUR ""LEIS"""  |
|I CAN ON1Y P13AS3 ON3 P3RSON MUG   |I CAN ONLY PLEASE ONE PERSON MUG   |
|HAND WARM3R BABUSHKA D3SIGN        |HAND WARMER BABUSHKA DESIGN        |
|B1U3 HARMONICA IN BOX              |B

In [9]:
import org.apache.spark.sql.functions.regexp_extract

// 처음 나타난 색상 이름을 추출하는 작업
val regexString = simpleColors.map(_.toUpperCase).mkString("(","|",")")

df.select(regexp_extract(col("Description"), regexString, 1).alias("color_clean"), col("Description")).show(50, false)

+-----------+-----------------------------------+
|color_clean|Description                        |
+-----------+-----------------------------------+
|           |HAWAIIAN GRASS SKIRT               |
|           |CHILLI LIGHTS                      |
|           |"RECORD FRAME 7"" SINGLE SIZE "    |
|           |3D DOG PICTURE PLAYING CARDS       |
|           |60 CAKE CASES VINTAGE CHRISTMAS    |
|           |PAPER CHAIN KIT VINTAGE CHRISTMAS  |
|           |CHRISTMAS TOILET ROLL              |
|           |"ASSORTED FLOWER COLOUR ""LEIS"""  |
|           |I CAN ONLY PLEASE ONE PERSON MUG   |
|           |HAND WARMER BABUSHKA DESIGN        |
|           |BLUE HARMONICA IN BOX              |
|null       |null                               |
|           |SET OF 72 RETROSPOT PAPER  DOILIES |
|           |PACK OF 72 RETROSPOT CAKE CASES    |
|           |WOODLAND DESIGN  COTTON TOTE BAG   |
|           |BIG DOUGHNUT FRIDGE MAGNETS        |
|           |RECYCLED PENCIL WITH RABBIT ERASER |


regexString = (BLACK|WHITE|RED|GREEN|BLUD)


(BLACK|WHITE|RED|GREEN|BLUD)

In [10]:
// 값 추출이 아닌 값의 존재 여부 확인
val containsBlack = col("Description").contains("BLACK")
val containsWhite = col("Description").contains("WHITE")

df.withColumn("hasSimpleColor", containsBlack.or(containsWhite))
    .where("hasSimpleColor").select("Description").show(5, false)

+----------------------------------+
|Description                       |
+----------------------------------+
|BAKING MOULD EASTER EGG WHITE CHOC|
|WHITE HANGING HEART T-LIGHT HOLDER|
|EDWARDIAN PARASOL BLACK           |
|BLACK EAR MUFF HEADPHONES         |
|BLACK RECORD COVER FRAME          |
+----------------------------------+
only showing top 5 rows



containsBlack = contains(Description, BLACK)
containsWhite = contains(Description, WHITE)


contains(Description, WHITE)

In [17]:
// 동적으로 인수의 개수가 변하는 경우 (???)
/// ???????????????????????????????????????????????????????????????
val simpleColors = Seq("black","white","red","green","blud")
val selectedColumns = simpleColors.map(color => {
    col("Description").contains(color.toUpperCase).alias(s"is_$color")
}):+expr("*")

df.select(selectedColumns:_*).where(col("is_white").or(col("is_red"))).select("Description").show(5, false)

Name: Unknown Error
Message: lastException: Throwable = null
<console>:36: error: not found: value expr
       }):+expr("*")
           ^

StackTrace: 

### 6. 날짜와 타임스탬프 데이터 다루기

날짜와 시간은 프로그래밍 언어와 데이터베이스 분야의 변함없는 과제이다. <br>
계속해 시간대를 확인해야 하며, 포맷이 올바르고 유효한지 확인해야 한다. <br>
이러한 복잡함을 피하고자 두 종류의 시간 관련 정보만 집중적으로 관리함. (달력 형태의 data, 날짜, 시간 정보를 가지는 timestamp) <br>
스파크 inferSchema 옵션이 활성화된 경우 날짜와 타임스탬프를 포함하 컬럼을 최대한 정확히 식별하려 노력함.

In [18]:
import org.apache.spark.sql.functions.{current_date, current_timestamp}

val dateDF = sc.range(10)
    .withColumn("today", current_date())
    .withColumn("now", current_timestamp())
dateDF.createOrReplaceTempView("dataTable")
dateDF.show(false)

+---+----------+-----------------------+
|id |today     |now                    |
+---+----------+-----------------------+
|0  |2019-07-26|2019-07-26 11:25:37.561|
|1  |2019-07-26|2019-07-26 11:25:37.561|
|2  |2019-07-26|2019-07-26 11:25:37.561|
|3  |2019-07-26|2019-07-26 11:25:37.561|
|4  |2019-07-26|2019-07-26 11:25:37.561|
|5  |2019-07-26|2019-07-26 11:25:37.561|
|6  |2019-07-26|2019-07-26 11:25:37.561|
|7  |2019-07-26|2019-07-26 11:25:37.561|
|8  |2019-07-26|2019-07-26 11:25:37.561|
|9  |2019-07-26|2019-07-26 11:25:37.561|
+---+----------+-----------------------+



dateDF = [id: bigint, today: date ... 1 more field]


[id: bigint, today: date ... 1 more field]

In [16]:
import org.apache.spark.sql.functions.{date_add, date_sub}

// 날짜 계산 5일 전후로.
dateDF.select(date_sub(col("today"), 5), date_add(col("today"), 5)).show(1)

+------------------+------------------+
|date_sub(today, 5)|date_add(today, 5)|
+------------------+------------------+
|        2019-07-21|        2019-07-31|
+------------------+------------------+
only showing top 1 row



In [21]:
import org.apache.spark.sql.functions.{datediff, months_between, to_date}

// 날짜간 일 차이 계산.
dateDF.withColumn("week_ago", date_sub(col("today"), 7))
    .select(datediff(col("week_ago"), col("today"))).show(1)

// 날짜간 월 차이 계산
// to_date는 문자열을 날짜로 변환할 수 있으며, 필요에 따라 날짜 포맷도 지정할 수 있음.
// 함수의 날짜 포맷은 자바의 SimpleDataFormat 클래스가 지원하는 포맷을 사용해야 함.
dateDF.select(
    to_date(lit("2016-01-01")).alias("start"),
    to_date(lit("2017-05-22")).alias("end")
).select(months_between(col("start"), col("end"))).show(1)

+-------------------------+
|datediff(week_ago, today)|
+-------------------------+
|                       -7|
+-------------------------+
only showing top 1 row

+--------------------------------+
|months_between(start, end, true)|
+--------------------------------+
|                    -16.67741935|
+--------------------------------+
only showing top 1 row



In [22]:
import org.apache.spark.sql.functions.{to_date, lit}

sc.range(5).withColumn("date", lit("2017-01-01"))
    .select(to_date(col("date"))).show(1)

+---------------+
|to_date(`date`)|
+---------------+
|     2017-01-01|
+---------------+
only showing top 1 row



In [24]:
// 파싱 할수 없는 날짜는 null을 출력 (yyyy-dd-MM) 이라 생각했을 때.
dateDF.select(to_date(lit("2016-20-12"))).show(1)
dateDF.select(to_date(lit("2017-12-11"))).show(1) // 얘는 에러라 판단하기가 힘듦.

+---------------------+
|to_date('2016-20-12')|
+---------------------+
|                 null|
+---------------------+
only showing top 1 row

+---------------------+
|to_date('2016-12-11')|
+---------------------+
|           2016-12-11|
+---------------------+
only showing top 1 row



In [23]:
// 위는 날짜 포맷을 입혀서 처리할 것.
import org.apache.spark.sql.functions.to_date

val dateFormat = "yyyy-dd-MM"
val cleanDateDF = sc.range(1).select(
    to_date(lit("2017-12-11"), dateFormat).alias("date"),
    to_date(lit("2017-20-12"), dateFormat).alias("date2")
)
cleanDateDF.show()

+----------+----------+
|      date|     date2|
+----------+----------+
|2017-01-12|2017-01-20|
+----------+----------+



dateFormat = yyyy-dd-mm
cleanDateDF = [date: date, date2: date]


[date: date, date2: date]

In [30]:
// timestamp는 항상 날짜 포맷을 지정해야함.
import org.apache.spark.sql.functions.to_timestamp

cleanDateDF.select(to_timestamp(col("date"), dateFormat)).show()

+----------------------------------+
|to_timestamp(`date`, 'yyyy-dd-MM')|
+----------------------------------+
|               2017-11-12 00:00:00|
+----------------------------------+



In [31]:
// 올바른 포맷과 타입의 날짜나 타임스탬프를 사용한다면 매우 쉽게 비교 연산도 가능
cleanDateDF.filter(col("date2") > lit("2017-12-12")).show()

+----------+----------+
|      date|     date2|
+----------+----------+
|2017-11-12|2017-12-20|
+----------+----------+



### 7. 널값 다루기
Dataframe에서는 빠져있거나 비어있는 데이터는 항상 null을 사용하는것이 좋음. <br>
빈 문자열이나 대체 값 대신 null값을 사용해야 최적화를 수행할 수 있음. <br>
Dataframe에 하위 패키지인 .na를 사용하는 것이 null을 다룰 수 있는 기본 방식.

In [34]:
// coalesce
// 인수로 지정한 여러 컬럼 중 null이 아닌 첫번 째 값을 반환함.
import org.apache.spark.sql.functions.coalesce

df.select(coalesce(col("Description"), col("CustomerId"))).show(20, false)
df.select(col("Description"), col("CustomerId")).show(20, false)

+----------------------------------+
|coalesce(Description, CustomerId) |
+----------------------------------+
|HAWAIIAN GRASS SKIRT              |
|CHILLI LIGHTS                     |
|"RECORD FRAME 7"" SINGLE SIZE "   |
|3D DOG PICTURE PLAYING CARDS      |
|60 CAKE CASES VINTAGE CHRISTMAS   |
|PAPER CHAIN KIT VINTAGE CHRISTMAS |
|CHRISTMAS TOILET ROLL             |
|"ASSORTED FLOWER COLOUR ""LEIS""" |
|I CAN ONLY PLEASE ONE PERSON MUG  |
|HAND WARMER BABUSHKA DESIGN       |
|BLUE HARMONICA IN BOX             |
|null                              |
|SET OF 72 RETROSPOT PAPER  DOILIES|
|PACK OF 72 RETROSPOT CAKE CASES   |
|WOODLAND DESIGN  COTTON TOTE BAG  |
|BIG DOUGHNUT FRIDGE MAGNETS       |
|RECYCLED PENCIL WITH RABBIT ERASER|
|RED TOADSTOOL LED NIGHT LIGHT     |
|RAIN PONCHO RETROSPOT             |
|STRAWBERRY CERAMIC TRINKET BOX    |
+----------------------------------+
only showing top 20 rows

+----------------------------------+----------+
|Description                       |Cu

ifnull: 첫번째 값이 null이면 두번째 값을 반환함, 첫번쨰 값이 null이 아니면 첫번쨰 값을 반환함. <br>
nullif: 두 값이 같으면 null을 반환함. 두 값이 다르면 첫번쨰 값을 반환함.<br>
nvl: ifnull 동일<br>
nvl2: 첫번째 값이 null이면 세번째 인수로 지정한 값을 반환함. 첫번째 값이 null이 아니면 두번째 값을 반환함. <br>
<br>
SELECT<br>
 ifnull(null, 'return_value'),<br>
 nullif('value', 'value'),<br>
 nvl(null, 'return_value'),<br>
 nvl2('not_null', 'return_value', 'else_value')<br>
<br>
a--> return_value<br>
b-->null<br>
c-->return_value<br>
d-->return_value<br>

In [37]:
// drop: null값을 가진 로우를 제거하는 함수
df.na.drop() 
df.na.drop("any") // 로우의 컬럼 중 하나라도 null이라면 해당 로우를 제거함.
df.na.drop("all") // 모든 컬럼의 값이 null이거나 NaN 인 경우에만 로우를 제거함.

df.na.drop("all", Seq("StockCode", "InvoiceNo")) // 특정 컬럼을 인수로 전달할 수 있음.

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   538172|    21562|HAWAIIAN GRASS SK...|      12|2010-12-10 09:33:00|     1.25|   15805.0|United Kingdom|
|   538172|    79321|       CHILLI LIGHTS|       8|2010-12-10 09:33:00|     4.95|   15805.0|United Kingdom|
|   538172|    22041|"RECORD FRAME 7""...|      12|2010-12-10 09:33:00|     2.55|   15805.0|United Kingdom|
|   538172|   84558A|3D DOG PICTURE PL...|      12|2010-12-10 09:33:00|     2.95|   15805.0|United Kingdom|
|   538172|    22952|60 CAKE CASES VIN...|      24|2010-12-10 09:33:00|     0.55|   15805.0|United Kingdom|
|   538172|    22910|PAPER CHAIN KIT V...|      24|2010-12-10 09:33:00|     2.95|   15805.0|United Kingdom|
|   538172|    21098|CHRISTM

In [38]:
// fill: 하나 이상의 컬럼을 특정 값으로 채울 수 있음.
df.na.fill("All Null values become this string")
df.na.fill(5, Seq("StockCode", "InvoiceNo")) // 특정 컬럼을 인수로 받을 수 있음.

[InvoiceNo: string, StockCode: string ... 6 more fields]

In [39]:
// Map 타입을 사용해 다수의 컬럼에 fill 메서드를 적용함
val fillColValues = Map("StockCode" -> 5, "Description" -> "No Value")
df.na.fill(fillColValues)

fillColValues = Map(StockCode -> 5, Description -> No Value)


[InvoiceNo: string, StockCode: string ... 6 more fields]

In [24]:
// replace: fill메서드 외에 null값을 유연하게 대처하기 위함. 조건에 따라 다른 값으로 대체함
df.na.replace("Description", Map("" -> "UNKNOWN")).show(5, false)

+---------+---------+-------------------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                    |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |
+---------+---------+-------------------------------+--------+-------------------+---------+----------+--------------+
|538172   |21562    |HAWAIIAN GRASS SKIRT           |12      |2010-12-10 09:33:00|1.25     |15805.0   |United Kingdom|
|538172   |79321    |CHILLI LIGHTS                  |8       |2010-12-10 09:33:00|4.95     |15805.0   |United Kingdom|
|538172   |22041    |"RECORD FRAME 7"" SINGLE SIZE "|12      |2010-12-10 09:33:00|2.55     |15805.0   |United Kingdom|
|538172   |84558A   |3D DOG PICTURE PLAYING CARDS   |12      |2010-12-10 09:33:00|2.95     |15805.0   |United Kingdom|
|538172   |22952    |60 CAKE CASES VINTAGE CHRISTMAS|24      |2010-12-10 09:33:00|0.55     |15805.0   |United Kingdom|
+---------+---------+---------------------------

### 8. 정렬하기

5장에서 설명한 것처럼 acs_nulls_first, desc_nulls_first, acs_nulls,last, desc_nulls_last 함수를 사용해 Dataframe 을 정렬할 때 null값이 표시되는 기준으로 지정할 수 있음.

### 9. 복합 데이터 타입 다루기

복합 데이터 타입은 구조체, 배열, 맵이 있음.

In [44]:
// 구조체: Dataframe 내부의 Dataframe으로 생각할 수 있음.
import org.apache.spark.sql.functions.struct

val complexDF = df.select(struct("Description", "InvoiceNo").alias("complex"))
complexDF.createOrReplaceTempView("complexDF")
complexDF.show(10, false)

+-------------------------------------------+
|complex                                    |
+-------------------------------------------+
|[HAWAIIAN GRASS SKIRT , 538172]            |
|[CHILLI LIGHTS, 538172]                    |
|["RECORD FRAME 7"" SINGLE SIZE ", 538172]  |
|[3D DOG PICTURE PLAYING CARDS, 538172]     |
|[60 CAKE CASES VINTAGE CHRISTMAS, 538172]  |
|[PAPER CHAIN KIT VINTAGE CHRISTMAS, 538172]|
|[CHRISTMAS TOILET ROLL, 538172]            |
|["ASSORTED FLOWER COLOUR ""LEIS""", 538172]|
|[I CAN ONLY PLEASE ONE PERSON MUG, 538172] |
|[HAND WARMER BABUSHKA DESIGN, 538172]      |
+-------------------------------------------+
only showing top 10 rows



complexDF = [complex: struct<Description: string, InvoiceNo: string>]


[complex: struct<Description: string, InvoiceNo: string>]

In [47]:
// 점을 사용해 getField 메서드를 사용할 수 있음.
complexDF.select("complex.Description").show(10, false)
complexDF.select(col("complex").getField("Description")).show(10, false)

+---------------------------------+
|Description                      |
+---------------------------------+
|HAWAIIAN GRASS SKIRT             |
|CHILLI LIGHTS                    |
|"RECORD FRAME 7"" SINGLE SIZE "  |
|3D DOG PICTURE PLAYING CARDS     |
|60 CAKE CASES VINTAGE CHRISTMAS  |
|PAPER CHAIN KIT VINTAGE CHRISTMAS|
|CHRISTMAS TOILET ROLL            |
|"ASSORTED FLOWER COLOUR ""LEIS"""|
|I CAN ONLY PLEASE ONE PERSON MUG |
|HAND WARMER BABUSHKA DESIGN      |
+---------------------------------+
only showing top 10 rows

+---------------------------------+
|complex.Description              |
+---------------------------------+
|HAWAIIAN GRASS SKIRT             |
|CHILLI LIGHTS                    |
|"RECORD FRAME 7"" SINGLE SIZE "  |
|3D DOG PICTURE PLAYING CARDS     |
|60 CAKE CASES VINTAGE CHRISTMAS  |
|PAPER CHAIN KIT VINTAGE CHRISTMAS|
|CHRISTMAS TOILET ROLL            |
|"ASSORTED FLOWER COLOUR ""LEIS"""|
|I CAN ONLY PLEASE ONE PERSON MUG |
|HAND WARMER BABUSHKA DESIGN      |
+-

In [49]:
// 배열
// split을 통해 배열로 변환함.
import org.apache.spark.sql.functions.split

df.select(split(col("Description"), " ")).show(5, false)

+--------------------------------------+
|split(Description,  )                 |
+--------------------------------------+
|[HAWAIIAN, GRASS, SKIRT, ]            |
|[CHILLI, LIGHTS]                      |
|["RECORD, FRAME, 7"", SINGLE, SIZE, "]|
|[3D, DOG, PICTURE, PLAYING, CARDS]    |
|[60, CAKE, CASES, VINTAGE, CHRISTMAS] |
+--------------------------------------+
only showing top 5 rows



In [51]:
df.select(split(col("Description"), " ").alias("array_col"))
    .selectExpr("array_col[0]").show(5, false)

+------------+
|array_col[0]|
+------------+
|HAWAIIAN    |
|CHILLI      |
|"RECORD     |
|3D          |
|60          |
+------------+
only showing top 5 rows



In [52]:
// 배열의 길이 (size 함수)
import org.apache.spark.sql.functions.size

df.select(size(split(col("Description"), " "))).show(5, false)

+---------------------------+
|size(split(Description,  ))|
+---------------------------+
|4                          |
|2                          |
|6                          |
|5                          |
|5                          |
+---------------------------+
only showing top 5 rows



In [59]:
// 배열에 특정 값이 존재하는지 확인 (array_contains 함수)
import org.apache.spark.sql.functions.array_contains

df.select(array_contains(split(col("Description"), " "), "WHITE")).show(5, false)

+--------------------------------------------+
|array_contains(split(Description,  ), WHITE)|
+--------------------------------------------+
|false                                       |
|false                                       |
|false                                       |
|false                                       |
|false                                       |
+--------------------------------------------+
only showing top 5 rows



In [62]:
// explode: 배열 타입의 컬럼을 입력 받음. 입력된 컬럼의 배열 값에 포함된 모든 값을 로우로 변환함.
import org.apache.spark.sql.functions.{split, explode}

df.withColumn("splitted", split(col("Description"), " "))
    .withColumn("exploded", explode(col("splitted")))
    .select("Description", "splitted", "exploded").show(5, false)

+---------------------+--------------------------+--------+
|Description          |splitted                  |exploded|
+---------------------+--------------------------+--------+
|HAWAIIAN GRASS SKIRT |[HAWAIIAN, GRASS, SKIRT, ]|HAWAIIAN|
|HAWAIIAN GRASS SKIRT |[HAWAIIAN, GRASS, SKIRT, ]|GRASS   |
|HAWAIIAN GRASS SKIRT |[HAWAIIAN, GRASS, SKIRT, ]|SKIRT   |
|HAWAIIAN GRASS SKIRT |[HAWAIIAN, GRASS, SKIRT, ]|        |
|CHILLI LIGHTS        |[CHILLI, LIGHTS]          |CHILLI  |
+---------------------+--------------------------+--------+
only showing top 5 rows



In [63]:
// Map
import org.apache.spark.sql.functions.map

df.select(map(col("Description"), col("InvoiceNo")).alias("complex_map")).show(5, false)

+-------------------------------------------+
|complex_map                                |
+-------------------------------------------+
|[HAWAIIAN GRASS SKIRT  -> 538172]          |
|[CHILLI LIGHTS -> 538172]                  |
|["RECORD FRAME 7"" SINGLE SIZE " -> 538172]|
|[3D DOG PICTURE PLAYING CARDS -> 538172]   |
|[60 CAKE CASES VINTAGE CHRISTMAS -> 538172]|
+-------------------------------------------+
only showing top 5 rows



In [69]:
// 적합한 키를 사용해 데이터를 조회할 수 있다. 없다면 null을 반환
df.select(map(col("Description"), col("InvoiceNo")).alias("complex_map"))
    .selectExpr("complex_map['HAWAIIAN GRASS SKIRT ']").show(5, false)

+----------------------------------+
|complex_map[HAWAIIAN GRASS SKIRT ]|
+----------------------------------+
|538172                            |
|null                              |
|null                              |
|null                              |
|null                              |
+----------------------------------+
only showing top 5 rows



In [71]:
// map을 분해해 컬럼으로 변환할 수 있음.
df.select(map(col("Description"), col("InvoiceNo")).alias("complex_map"))
.selectExpr("explode(complex_map)").show(2, false)

+---------------------+------+
|key                  |value |
+---------------------+------+
|HAWAIIAN GRASS SKIRT |538172|
|CHILLI LIGHTS        |538172|
+---------------------+------+
only showing top 2 rows



### 10. Json 다루기

json을 위한 몇가지 고유 기능을 지원함. <br>
문자열 형태의 json을 직접 조작할 수 있으며, json 파싱, json 객체로 만들 수 있음.

In [72]:
val jsonDF = sc.range(1).selectExpr("""
    '{"myJsonKey": {"myJsonValue":[1, 2, 3]}}' as jsonString """
)

jsonDF = [jsonString: string]


[jsonString: string]

In [75]:
import org.apache.spark.sql.functions.{get_json_object, json_tuple}

jsonDF.select(
    get_json_object(col("jsonString"), "$.myJsonKey.myJsonValue[1]") as "column", 
    json_tuple(col("jsonString"), "myJsonKey")
).show(2, false)

+------+-----------------------+
|column|c0                     |
+------+-----------------------+
|2     |{"myJsonValue":[1,2,3]}|
+------+-----------------------+



In [77]:
// to_json을 통해 structType을 JSON 문자열로 변경할 수 있음
import org.apache.spark.sql.functions.to_json

df.selectExpr("(InvoiceNo, Description) as myStruct")
    .select(to_json(col("myStruct"))).show(2, false)

+------------------------------------------------------------+
|structstojson(myStruct)                                     |
+------------------------------------------------------------+
|{"InvoiceNo":"538172","Description":"HAWAIIAN GRASS SKIRT "}|
|{"InvoiceNo":"538172","Description":"CHILLI LIGHTS"}        |
+------------------------------------------------------------+
only showing top 2 rows



In [78]:
// to_json 함수에 JSON 데이터 소스와 동일한 형태의 딕셔너리를 파라미터로 사용할 수 있음
// 그리고 from_json 함수를 사용해 JSON 문자열을 다시 객체로 변환할 수 있음
import org.apache.spark.sql.functions.from_json
import org.apache.spark.sql.types._

val parseSchema = new StructType ( Array(
    new StructField("InvoiceNo", StringType, true), 
    new StructField("Description", StringType, true)))

df.selectExpr("(InvoiceNo, Description) as myStruct")
    .select(to_json(col("myStruct")).alias("newJSON"))
    .select(from_json(col("newJSON"), parseSchema), col("newJSON")).show(5, false)

+-----------------------------------------+--------------------------------------------------------------------------+
|jsontostructs(newJSON)                   |newJSON                                                                   |
+-----------------------------------------+--------------------------------------------------------------------------+
|[538172, HAWAIIAN GRASS SKIRT ]          |{"InvoiceNo":"538172","Description":"HAWAIIAN GRASS SKIRT "}              |
|[538172, CHILLI LIGHTS]                  |{"InvoiceNo":"538172","Description":"CHILLI LIGHTS"}                      |
|[538172, "RECORD FRAME 7"" SINGLE SIZE "]|{"InvoiceNo":"538172","Description":"\"RECORD FRAME 7\"\" SINGLE SIZE \""}|
|[538172, 3D DOG PICTURE PLAYING CARDS]   |{"InvoiceNo":"538172","Description":"3D DOG PICTURE PLAYING CARDS"}       |
|[538172, 60 CAKE CASES VINTAGE CHRISTMAS]|{"InvoiceNo":"538172","Description":"60 CAKE CASES VINTAGE CHRISTMAS"}    |
+-----------------------------------------+-----

parseSchema = StructType(StructField(InvoiceNo,StringType,true), StructField(Description,StringType,true))


StructType(StructField(InvoiceNo,StringType,true), StructField(Description,StringType,true))

### 11. 사용자 정의 함수

스파크의 가장 강력한 기능중 하나로 사용자 정의 함수 (UDF)를 사용할 수 있다. <br>
UDF는 파이썬, 스칼라, 외부 라이브러리를 사용해 사용자가 원하는 형태로 트랜스포메이션을 만들 수 있게 함. <br>
<br>
UDF는 하나 이상의 컬럼을 입력으로 받고, 반환할 수 있음. <br>
레코드별로 데이터를 처리하는 함수이기 때문에 독특한 포맷이나 도메인에 특화된 언어를 사용하지 않음. <br>
SparkSession, Context에서 사용할 수 있도록 임시 함수 형태로 등록함 <br>

In [79]:
val udfExampleDF = sc.range(5).toDF("num")

def power3(number:Double):Double = number * number * number
power3(2.0)

udfExampleDF = [num: bigint]


power3: (number: Double)Double


8.0

위와 같이 함수를 만들고 테스트를 완료했으면, 실제 모든 워커 노드에서 생성된 함수를 사용할 수 있도록 스파크에 등록해야함 <br>
스파크는 드라이버에서 함수를 직렬화하고 네트워크를 통해 모든 익스큐터 프로세스로 전달하며, 이과정은 언어와 관계 없이 발생함. <br>
<br>
함수를 개발한 언어에 따라 근본적으로 동작하는 방식이 달라짐. <br>
스파크, 자바는 JVM환경에서만 사용할 수 있음. 따라서 스파크 내장 함수가 제공하는 코드 생성 기능의 장점을 활용할 수 없어 약간의 성능 저하가 발생함. <br>
파이썬은 워커 노드에 파이썬 프로세스를 실행하고 파이썬이 이해할 수 있는 포맷으로 모든 데이터를 직렬화함. 그리고 파이썬 프로세스에 있는 모든 데이터의 로우마다 함수를 실행하고 마지막으로 JVM과 스파크에 처리 결과를 반환함. <br>
<br>
파이썬의 문제점으로 <br>
1. 직렬화에 큰 부하가 발생함. (이는 스파크, 자바에서도 동일하게 발생)
2. 스파크에서 파이썬 프로세스 실행 후 데이터를 전송하는데, 데이터가 전달되면 스파크에서 워커 메모리를 관리할 수 없음. 그러므로 JVM과 파이썬이 동일한 머신에서 메모리 경합을 하면 자원에 제약이 생겨 워커가 비정상적으로 종료될 가능성이 있음. 
3. 결론은 되도록이면 UDF는 자바나 스칼라로 작성하는 것이 좋다.

In [80]:
import org.apache.spark.sql.functions.udf

// 사용자 정의 함수는 Dataframe에서만 사용 가능. 문자열 표현식에서는 사용할 수 없다.
// 하지만, 사용자 정의 함수를 스파크 SQL 함수로 등록하면 사용이 가능하다.
val power3udf = udf(power3(_:Double):Double)
udfExampleDF.select(power3udf(col("num"))).show(5, false)

+--------+
|UDF(num)|
+--------+
|0.0     |
|1.0     |
|8.0     |
|27.0    |
|64.0    |
+--------+



power3udf = UserDefinedFunction(<function1>,DoubleType,Some(List(DoubleType)))


UserDefinedFunction(<function1>,DoubleType,Some(List(DoubleType)))

### 12. Hive UDF
하이브 문법을 사용해 UDF/UDAF도 사용할 수 있다. <br>
하지만 이렇게 하려면 SparkSession을 생성할 때, SparkSession.builder().enableHiveSupport()를 명시해 반드시 하이브 지원 기능을 활성화 해야함. <br>
이 후 SQL로 UDF를 등록할 수 있다. <br>

```
-- SQL
CREATE TEMPORARY FUNCTION myFunc AS 'com.organization.hive.udf.FunctionName'
```